### Create AML SDK docker file and build and test image

To run this notebook, start it via a plain Jupyter notebook session

* login (ssh) into the VM:
```
loginvm0011@ghiordanXRgpuvm:/datadrive01/prj/regularR_Realtime$ jupyter notebook --notebook-dir=$(pwd) --ip='*' --port=9000 --no-browser --allow-root
```



  
   
* then go to:    
http://ghiordanxrgpuvm.eastus2.cloudapp.azure.com:9000/ 
  
  
Note: continuumio/miniconda3:4.5.4 is the latest tag as of 9/7/2018

In [1]:
# Allow multiple displays per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [2]:
%load_ext dotenv
# import dotenv
# print (dir(dotenv))

In [3]:
%%writefile .env

# Your docker login and image repository name
docker_login = 'georgedockeraccount'
image_tag = "aml-sdk_docker_image"
image_version = 'sdk.v.1.0.10' 

Overwriting .env


In [4]:
%dotenv
docker_file_location = 'docker'
conda_dependency_file = 'aml_sdk_conda_dep_file.yml'

import os
docker_file_name = 'dockerfile'+ '_' + os.getenv('image_tag') + '_' + os.getenv('image_version')
docker_file_name

'dockerfile_aml-sdk_docker_image_sdk.v.1.0.10'

In [5]:
local_dir = !pwd
docker_file_dir = os.path.join(*([local_dir.s, docker_file_location]))
docker_file_dir
 
docker_file_path = os.path.join(*([docker_file_dir]+[docker_file_name]))
docker_file_path

conda_dependency_file_path = os.path.join(*([docker_file_dir]+[conda_dependency_file]))
conda_dependency_file_path

!mkdir -p {docker_file_dir}
! chmod -R ugo=rwx $docker_file_dir
!rm -rf $docker_file_dir/*
!ls -l $docker_file_dir

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker'

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker/dockerfile_aml-sdk_docker_image_sdk.v.1.0.10'

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker/aml_sdk_conda_dep_file.yml'

total 0


In [6]:
%%writefile $conda_dependency_file_path
name: aml_sdk_conda_env02
    
channels:
  - anaconda
dependencies:
  - python=3.6 
  - numpy
  - cython
  - notebook=5.6.0 
  - nb_conda
  - scikit-learn
  - pip:
    - python-dotenv
    - azureml-sdk[notebooks,automl,explain] 

Writing /datadrive01/prj/PowerAIWithDocker/amlsdk/docker/aml_sdk_conda_dep_file.yml


In [7]:
# %%writefile $docker_file_path 

# FROM debian:latest
# MAINTAINER George Iordanescu <ghiordan@microsoft.com>
# # https://hub.docker.com/r/continuumio/miniconda3/~/dockerfile/

# ENV LANG=C.UTF-8 LC_ALL=C.UTF-8
# ENV PATH /opt/conda/bin:$PATH

# RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends \
#     wget bzip2 ca-certificates curl git && \
#     apt-get clean && \
#     rm -rf /var/lib/apt/lists/*
    
# # # Install Azure CLI
# # RUN echo "deb [arch=amd64] https://packages.microsoft.com/repos/azure-cli/ xenial main" | \
# #     tee /etc/apt/sources.list.d/azure-cli.list && \
# #     curl -L https://packages.microsoft.com/keys/microsoft.asc | apt-key add - && \
# #     apt-get update && \
# #     apt-get install -y --no-install-recommends \
# #     azure-cli


# ADD aml_sdk_conda_dep_file.yml /tmp/aml_sdk_conda_dep_file.yml

# ENV CONDA_ENV_NAME aml-sdk-conda-env

# # RUN curl -o ~/miniconda.sh -O  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh  && \
# RUN wget --quiet https://repo.anaconda.com/miniconda/Miniconda3-4.5.11-Linux-x86_64.sh -O ~/miniconda.sh && \
#     chmod +x ~/miniconda.sh && \
#     ~/miniconda.sh -b -p /opt/conda && \
#     rm ~/miniconda.sh 
    
# RUN  conda env create -q --name $CONDA_ENV_NAME -f /tmp/aml_sdk_conda_dep_file.yml && \
#     conda clean --yes --all
# ENV PATH /opt/conda/envs/$CONDA_ENV_NAME/bin:$PATH

# RUN [ "/bin/bash","-c","source activate aml-sdk-conda-env && pip install azureml-sdk[notebooks]"]    
# # install and enable experiment widget
# # https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-environment
# RUN ["/bin/bash","-c","source activate aml-sdk-conda-env && \
#                     jupyter nbextension install --py --user azureml.train.widgets  && \
#                     jupyter nbextension enable --py --user azureml.train.widgets"]

# WORKDIR /workspace

# CMD [ "/bin/bash" ]

In [8]:
# %%writefile $docker_file_path 

# FROM continuumio/miniconda3:4.5.11
# MAINTAINER George Iordanescu <ghiordan@microsoft.com>

# RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends \
#     wget curl  \
#     git && \ 
#     apt-get clean && \
#     rm -rf /var/lib/apt/lists/*
    
# ADD aml_sdk_conda_dep_file.yml /tmp/aml_sdk_conda_dep_file.yml

# ENV CONDA_ENV_NAME aml-sdk-conda-env

# RUN  conda env create -q --name $CONDA_ENV_NAME -f /tmp/aml_sdk_conda_dep_file.yml && \
#     conda clean --yes --all
# ENV PATH /opt/conda/envs/$CONDA_ENV_NAME/bin:$PATH

# #unclear if I can run pip from $CONDA_ENV_NAME env, but     
# RUN [ "/bin/bash","-c","source activate aml-sdk-conda-env && pip install azureml-sdk[notebooks,automl,explain]"] 

# # install and enable experiment widget
# # https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-environment
# RUN [ "/bin/bash","-c","source activate aml-sdk-conda-env && \
#                         jupyter nbextension install --py --user azureml.widgets  && \
#                         jupyter nbextension enable --py --user azureml.widgets"]

# # RUN [ "/bin/bash","-c","source activate aml-sdk-conda-env && \
# #                         mkdir -p /workspace"]

# CMD [ "/bin/bash","-c","source activate aml-sdk-conda-env"]

In [9]:
%%writefile $docker_file_path 

# https://hub.docker.com/_/ubuntu?tab=tags
FROM ubuntu:19.04
MAINTAINER George Iordanescu <ghiordan@microsoft.com>

# basic system packages, plus gcc
RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends \
    gcc g++ \
    wget bzip2 ca-certificates curl git && \
    apt-get clean && \
    rm -rf /var/lib/apt/lists/*

#install miniconda and create a single conda env based on our .yml file
ADD aml_sdk_conda_dep_file.yml /tmp/aml_sdk_conda_dep_file.yml
ENV CONDA_DIR /opt/conda
ENV PATH $CONDA_DIR/bin:$PATH
ENV CONDA_ENV_NAME aml-sdk-conda-env    
RUN wget --quiet --no-check-certificate https://repo.continuum.io/miniconda/Miniconda3-4.2.12-Linux-x86_64.sh && \
    echo "c59b3dd3cad550ac7596e0d599b91e75d88826db132e4146030ef471bb434e9a *Miniconda3-4.2.12-Linux-x86_64.sh" | sha256sum -c - && \
    /bin/bash /Miniconda3-4.2.12-Linux-x86_64.sh -f -b -p $CONDA_DIR && \
    rm Miniconda3-4.2.12-Linux-x86_64.sh && \
    conda env create -q --name $CONDA_ENV_NAME -f /tmp/aml_sdk_conda_dep_file.yml && \
    conda clean  --yes --all && \
    ln -s $CONDA_DIR/etc/profile.d/conda.sh /etc/profile.d/conda.sh && \
    echo ". /opt/conda/etc/profile.d/conda.sh" >> ~/.bashrc && \
    echo "conda activate $CONDA_ENV_NAME" >> ~/.bashrc
ENV PATH $CONDA_DIR/envs/$ENV_NAME/bin:$PATH
    
#unclear if I can run pip from $CONDA_ENV_NAME env, but     
# RUN [ "/bin/bash","-c","pip install azureml-sdk[notebooks,automl]"] 

# # install and enable experiment widget
# # https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-environment
# RUN [ "/bin/bash","-c","jupyter nbextension install --py --user azureml.widgets  && \
#                         jupyter nbextension enable --py --user azureml.widgets"]        

CMD /bin/bash

Writing /datadrive01/prj/PowerAIWithDocker/amlsdk/docker/dockerfile_aml-sdk_docker_image_sdk.v.1.0.10


In [10]:
%dotenv

#add docker image version info only if image_version has been set to a non-empty string, otherwise let docker increment it
image_version_string = os.getenv('image_version')
if image_version_string!="":
    image_version_string = ':'  + image_version_string
    
docker_image_name = os.getenv('docker_login') + '/' + os.getenv('image_tag') + image_version_string


docker_file_path
docker_image_name
docker_file_dir

working_path = docker_file_dir
! ls -l $working_path


'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker/dockerfile_aml-sdk_docker_image_sdk.v.1.0.10'

'georgedockeraccount/aml-sdk_docker_image:sdk.v.1.0.10'

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker'

total 8
-rw-rw-r-- 1 loginvm001 loginvm001  228 Feb  9 03:31 aml_sdk_conda_dep_file.yml
-rw-rw-r-- 1 loginvm001 loginvm001 1774 Feb  9 03:31 dockerfile_aml-sdk_docker_image_sdk.v.1.0.10


In [11]:
!echo docker build -t $docker_image_name -f $docker_file_path $working_path --no-cache
!     docker build -t $docker_image_name -f $docker_file_path $working_path --no-cache

docker build -t georgedockeraccount/aml-sdk_docker_image:sdk.v.1.0.10 -f /datadrive01/prj/PowerAIWithDocker/amlsdk/docker/dockerfile_aml-sdk_docker_image_sdk.v.1.0.10 /datadrive01/prj/PowerAIWithDocker/amlsdk/docker --no-cache
Sending build context to Docker daemon  4.608kB
Step 1/10 : FROM ubuntu:19.04
 ---> d6e206581aca
Step 2/10 : MAINTAINER George Iordanescu <ghiordan@microsoft.com>
 ---> Running in 03457ac3a624
Removing intermediate container 03457ac3a624
 ---> affac32a8770
Step 3/10 : RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends     gcc g++     wget bzip2 ca-certificates curl git &&     apt-get clean &&     rm -rf /var/lib/apt/lists/*
 ---> Running in 9064208e4814
Get:1 http://security.ubuntu.com/ubuntu disco-security InRelease [65.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu disco InRelease [243 kB]
Get:3 http://archive.ubuntu.com/ubuntu disco-updates InRelease [65.4 kB]
Get:4 http://archive.ubuntu.com/ubuntu disco-backports InRelease [65.5 kB]


Get:41 http://archive.ubuntu.com/ubuntu disco/main amd64 libgssapi3-heimdal amd64 7.5.0+dfsg-2.1 [96.4 kB]
Get:42 http://archive.ubuntu.com/ubuntu disco/main amd64 libsasl2-modules-db amd64 2.1.27+dfsg-1 [14.9 kB]
Get:43 http://archive.ubuntu.com/ubuntu disco/main amd64 libsasl2-2 amd64 2.1.27+dfsg-1 [49.4 kB]
Get:44 http://archive.ubuntu.com/ubuntu disco/main amd64 libldap-common all 2.4.47+dfsg-2ubuntu1 [16.2 kB]
Get:45 http://archive.ubuntu.com/ubuntu disco/main amd64 libldap-2.4-2 amd64 2.4.47+dfsg-2ubuntu1 [154 kB]
Get:46 http://archive.ubuntu.com/ubuntu disco/main amd64 libnghttp2-14 amd64 1.36.0-2 [79.0 kB]
Get:47 http://archive.ubuntu.com/ubuntu disco/main amd64 librtmp1 amd64 2.4+20151223.gitfa8646d.1-2 [54.5 kB]
Get:48 http://archive.ubuntu.com/ubuntu disco/main amd64 libssh-4 amd64 0.8.6-2 [185 kB]
Get:49 http://archive.ubuntu.com/ubuntu disco/main amd64 libcurl4 amd64 7.63.0-1ubuntu1 [232 kB]
Get:50 http://archive.ubuntu.com/ubuntu disco/main amd64 curl amd64 7.63.0-1ubuntu

Selecting previously unselected package binutils.
Preparing to unpack .../17-binutils_2.32-1ubuntu4_amd64.deb ...
Unpacking binutils (2.32-1ubuntu4) ...
Selecting previously unselected package libisl19:amd64.
Preparing to unpack .../18-libisl19_0.20-2_amd64.deb ...
Unpacking libisl19:amd64 (0.20-2) ...
Selecting previously unselected package libmpfr6:amd64.
Preparing to unpack .../19-libmpfr6_4.0.2-1_amd64.deb ...
Unpacking libmpfr6:amd64 (4.0.2-1) ...
Selecting previously unselected package libmpc3:amd64.
Preparing to unpack .../20-libmpc3_1.1.0-1_amd64.deb ...
Unpacking libmpc3:amd64 (1.1.0-1) ...
Selecting previously unselected package cpp-8.
Preparing to unpack .../21-cpp-8_8.2.0-19ubuntu1_amd64.deb ...
Unpacking cpp-8 (8.2.0-19ubuntu1) ...
Selecting previously unselected package cpp.
Preparing to unpack .../22-cpp_4%3a8.2.0-2ubuntu2_amd64.deb ...
Unpacking cpp (4:8.2.0-2ubuntu2) ...
Selecting previously unselected package libroken18-heimdal:amd64.
Preparing to unpack .../23-librok

Setting up libgomp1:amd64 (9-20190207-0ubuntu1) ...
Setting up libatomic1:amd64 (9-20190207-0ubuntu1) ...
Setting up git-man (1:2.20.1-2ubuntu1) ...
Setting up libexpat1:amd64 (2.2.6-1) ...
Setting up libcc1-0:amd64 (9-20190207-0ubuntu1) ...
Setting up libnghttp2-14:amd64 (1.36.0-2) ...
Setting up libldap-common (2.4.47+dfsg-2ubuntu1) ...
Setting up libpsl5:amd64 (0.20.2-2) ...
Setting up libtsan0:amd64 (9-20190207-0ubuntu1) ...
Setting up libsasl2-modules-db:amd64 (2.1.27+dfsg-1) ...
Setting up linux-libc-dev:amd64 (4.19.0-12.13) ...
Setting up libmpfr6:amd64 (4.0.2-1) ...
Setting up libsasl2-2:amd64 (2.1.27+dfsg-1) ...
Setting up libroken18-heimdal:amd64 (7.5.0+dfsg-2.1) ...
Setting up librtmp1:amd64 (2.4+20151223.gitfa8646d.1-2) ...
Setting up libkrb5support0:amd64 (1.17-1) ...
Setting up liblsan0:amd64 (9-20190207-0ubuntu1) ...
Setting up libgdbm6:amd64 (1.18.1-2build1) ...
Setting up binutils-common:amd64 (2.32-1ubuntu4) ...
Setting up perl-modules-5.28 (5.28.1-4) ...
Processing t

  Stored in directory: /root/.cache/pip/wheels/45/cb/7e/ce6e6062c69446e39e328170524ca8213498bc66a74c6a210b
  Stored in directory: /root/.cache/pip/wheels/4e/5b/1b/be8c8a830a0243af85b2946a0aece2c6743d7f7f946977ed67
  Stored in directory: /root/.cache/pip/wheels/ad/da/0c/74eb680767247273e2cf2723482cb9c924fe70af57c334513f
  Stored in directory: /root/.cache/pip/wheels/5b/d7/0f/e58eae695403de585269f4e4a94e0cd6ca60ec0c202936fa4a
  Stored in directory: /root/.cache/pip/wheels/21/88/00/697879eab372b1071f60675adf91f6da0ae015313318b35421
  Stored in directory: /root/.cache/pip/wheels/ee/30/5c/3a3b5e1386c8db9a3be5f5c3933644ae0533c1351c6a8eb4b5
  Stored in directory: /root/.cache/pip/wheels/e8/74/51/42c2d41c02bdc6f0e604476b7e4293b8c98d0bcbfa1dff78c8
  Stored in directory: /root/.cache/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
  Stored in directory: /root/.cache/p

Enabling nb_conda_kernels...
Status: enabled
Enabling notebook extension nb_conda/main...
      - Validating: OK
Enabling tree extension nb_conda/tree...
      - Validating: OK
Enabling: nb_conda
- Writing config: /opt/conda/envs/aml-sdk-conda-env/etc/jupyter
    - Validating...
      nb_conda 2.2.1 OK
#
# To activate this environment, use:
# > source activate aml-sdk-conda-env
#
# To deactivate this environment, use:
# > source deactivate aml-sdk-conda-env
#

conda-build is not installed; could not clean source cache
Cache location: /opt/conda/pkgs
Will remove the following tarballs:

/opt/conda/pkgs
---------------
wcwidth-0.1.7-py36hdf4376a_0.tar.bz2          25 KB
decorator-4.3.2-py36_0.tar.bz2                17 KB
pickleshare-0.7.5-py36_0.tar.bz2              13 KB
numpy-1.15.4-py36h1d66e8a_0.tar.bz2           35 KB
ipython-7.2.0-py36h39e3cac_0.tar.bz2         1.0 MB
pygments-2.3.1-py36_0.tar.bz2                1.4 MB
webencodings-0.5.1-py36h800622e_1.tar.bz2      19 KB
nbformat-4

Removing intermediate container 51de8ba75413
 ---> 2a5a1d9c3d28
Step 9/10 : ENV PATH $CONDA_DIR/envs/$ENV_NAME/bin:$PATH
 ---> Running in a4be9642c0f5
Removing intermediate container a4be9642c0f5
 ---> fb0c5889004b
Step 10/10 : CMD /bin/bash
 ---> Running in 3e8c842b58ef
Removing intermediate container 3e8c842b58ef
 ---> 28704890b2c6
Successfully built 28704890b2c6
Successfully tagged georgedockeraccount/aml-sdk_docker_image:sdk.v.1.0.10


In [12]:
local_dir = !pwd
crt_local_dir = os.path.join(*([local_dir.s, 'docker_run_dir']))
crt_local_dir 
!mkdir -p $crt_local_dir
!rm -rf $crt_local_dir/test_*
!ls -l $crt_local_dir
# ! chmod -R ugo=rwx $crt_local_dir

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_run_dir'

total 0


In [13]:
!mkdir -p ./docker_history
!cp $docker_file_path ./docker_history/

#### Push image to ([dockerhub](https://hub.docker.com/)) registry (optional step)

In [14]:
# !docker login -u=$docker_login -p=

In [15]:
!docker push $docker_image_name

The push refers to repository [docker.io/georgedockeraccount/aml-sdk_docker_image]

5d288dd3: Preparing 
64c6a1ba: Preparing 
2ac369eb: Preparing 
b008113b: Preparing 
0d749dd9: Preparing 
b62d5c32: Preparing 


5d288dd3: Pushing  1.862GB/2.534GBPushing  127.6MB/258.7MB

5d288dd3: Pushed    2.59GB/2.534GBPushing  2.207GB/2.534GBsdk.v.1.0.10: digest: sha256:80478443bf2312a25146aed87bedee3ab456300e5d31d0002a48c66ce42c2b38 size: 1782


### To run aml SDK, ssh into a vm, run the command below and then go to: 
ghiordanxrgpuvm.eastus2.cloudapp.azure.com:9001/  
  
Make sure the host port (9001) is open in the VM.  

In [16]:
!echo docker run -it -p 9001:8888 -v $(pwd)/../:/workspace:rw $docker_image_name /bin/bash -c '"source activate aml-sdk-conda-env && jupyter notebook --notebook-dir=/workspace --ip='*' --port=8888 --no-browser --allow-root"'

docker run -it -p 9001:8888 -v /datadrive01/prj/PowerAIWithDocker/amlsdk/../:/workspace:rw georgedockeraccount/aml-sdk_docker_image:sdk.v.1.0.10 /bin/bash -c "source activate aml-sdk-conda-env && jupyter notebook --notebook-dir=/workspace --ip=* --port=8888 --no-browser --allow-root"


### Run a short test
Run ' import azureml.core; print("AML SDK version:", azureml.core.VERSION) ' in a notebook

In [17]:
test_notebook_base_name = 'test_aml_sdk_docker_image'
test_notebook_name = test_notebook_base_name+'.ipynb'

In [18]:
%%writefile $crt_local_dir/$test_notebook_name

{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "AML SDK version: 0.1.68\n"
     ]
    }
   ],
   "source": [
    "# Check core SDK version number\n",
    "import azureml.core\n",
    "\n",
    "print(\"AML SDK version:\", azureml.core.VERSION)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python [conda env:aml-sdk-conda-env]",
   "language": "python",
   "name": "conda-env-aml-sdk-conda-env-py"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.6.6"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}


Writing /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_run_dir/test_aml_sdk_docker_image.ipynb


In [19]:
!ls -l $crt_local_dir

total 4
-rw-rw-r-- 1 loginvm001 loginvm001 982 Feb  9 03:38 test_aml_sdk_docker_image.ipynb


### Quick testing SDK docker image (see cell response "AML SDK version: <latest version>").
Compare the SDK version against latest AML SDK [relese notes](https://docs.microsoft.com/en-us/azure/machine-learning/service/azure-machine-learning-release-notes)

In [20]:
!docker run -v $crt_local_dir:/workspace:rw $docker_image_name /bin/bash -c "source activate aml-sdk-conda-env && jupyter kernelspec list && jupyter nbconvert --ExecutePreprocessor.kernel_name=python3 --execute --to notebook  /workspace/test_aml_sdk_docker_image.ipynb"

Available kernels:
  python3    /opt/conda/envs/aml-sdk-conda-env/share/jupyter/kernels/python3
[NbConvertApp] Converting notebook /workspace/test_aml_sdk_docker_image.ipynb to notebook
[NbConvertApp] Executing notebook with kernel: python3
[NbConvertApp] Writing 979 bytes to /workspace/test_aml_sdk_docker_image.nbconvert.ipynb


In [21]:
output_file = test_notebook_base_name+'.nbconvert.ipynb'
!cat $crt_local_dir/$output_file

{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "AML SDK version: 1.0.10\n"
     ]
    }
   ],
   "source": [
    "# Check core SDK version number\n",
    "import azureml.core\n",
    "\n",
    "print(\"AML SDK version:\", azureml.core.VERSION)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python [conda env:aml-sdk-conda-env]",
   "language": "python",
   "name": "conda-env-aml-sdk-conda-env-py"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.6.6"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}


In [22]:
!jupyter nbconvert --to html createAMLSDKDocker_002.ipynb

[NbConvertApp] Converting notebook createAMLSDKDocker_002.ipynb to html
[NbConvertApp] Writing 368479 bytes to createAMLSDKDocker_002.html
